# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-5e443300e2-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("International Treaties") 

Consider the following exploratory information need:

> You investigating existing international treaties

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q131569`  | treaty | node      |
| `wd:Q46`      | Europe        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q21707860` | Paris Agreement           | node |



Also consider

```
?p wdt:P31/wdt:P279* wd:Q131569  . 
```

is the BGP to retrieve all **treaties**

## Workload Goals


1. Identify the BGP to retrieve treaties that took place in E.U. countries

2. Identify the BGP for obtaining which countries or people signed the treaties

3. How many treaties have been published per year?

4. For how many treaties we know causes or effects?

5. Analyze the number of countries and how many treaties they have signed and their causes
 
   5.1 Are there two countries that signed many common treaties?
   
   5.2 Which are the treaties with most signatories? Which country signed the most treaties?
   
   5.3 Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?


In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P31/wdt:P279* wd:Q131569  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '3962')]


1

## Part 1
### Identify the BGP to retrieve treaties that took place in E.U. countries

### Try to reach "took place" or similar predicate

In [4]:
queryString = """
SELECT DISTINCT ?pName ?p
WHERE { 

    ?s wdt:P31/wdt:P279* wd:Q131569  .
    ?s ?p ?o.
    OPTIONAL{?p sc:name ?pName}
    FILTER (regex(?pName, ".*place.*")||regex(?pName, ".*loca.*"))
    OPTIONAL{?o sc:name ?oName}
} LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pName', 'located in the administrative territorial entity'), ('p', 'http://www.wikidata.org/prop/direct/P131')]
[('pName', 'locator map image'), ('p', 'http://www.wikidata.org/prop/direct/P242')]
[('pName', 'location'), ('p', 'http://www.wikidata.org/prop/direct/P276')]
[('pName', 'located in/on physical feature'), ('p', 'http://www.wikidata.org/prop/direct/P706')]
[('pName', 'replaces'), ('p', 'http://www.wikidata.org/prop/direct/P1365')]
[('pName', 'replaced by'), ('p', 'http://www.wikidata.org/prop/direct/P1366')]
[('pName', 'place of publication'), ('p', 'http://www.wikidata.org/prop/direct/P291')]
[('pName', 'significant place'), ('p', 'http://www.wikidata.org/prop/direct/P7153')]
[('pName', 'headquarters location'), ('p', 'http://www.wikidata.org/prop/direct/P159')]
[('pName', 'location of formation'), ('p', 'http://www.wikidata.org/prop/direct/P740')]
[('pName', 'location of creation'), ('p', 'http://www.wikidata.org/prop/direct/P1071')]
[('pName', 'located in time zo

12

### See difference of "location" wdt:P276 and "place of pubblication" wdt:P291

In [5]:
queryString = """
SELECT DISTINCT ?location ?publ
WHERE { 

    ?s wdt:P31/wdt:P279* wd:Q131569 .
    
    OPTIONAL{ ?s wdt:P276 ?location . }
    OPTIONAL{ ?s wdt:P291 ?publ . }
    
} LIMIT 30
"""

print("Results")
run_query(queryString)

Results
[('location', 'http://www.wikidata.org/entity/Q71'), ('publ', 'http://www.wikidata.org/entity/Q71')]
[('publ', 'http://www.wikidata.org/entity/Q15681')]
[('publ', 'http://www.wikidata.org/entity/Q340')]
[('publ', 'http://www.wikidata.org/entity/Q20934')]
[('publ', 'http://www.wikidata.org/entity/Q98633573')]
[('publ', 'http://www.wikidata.org/entity/Q43631')]
[('publ', 'http://www.wikidata.org/entity/Q71')]
[('publ', 'http://www.wikidata.org/entity/Q1773')]
[('location', 'http://www.wikidata.org/entity/Q168057')]
[('location', 'http://www.wikidata.org/entity/Q1309')]
[('location', 'http://www.wikidata.org/entity/Q60')]
[('location', 'http://www.wikidata.org/entity/Q3844')]
[('location', 'http://www.wikidata.org/entity/Q1741')]
[('location', 'http://www.wikidata.org/entity/Q60')]
[('location', 'http://www.wikidata.org/entity/Q5465')]
[('location', 'http://www.wikidata.org/entity/Q211818')]
[('location', 'http://www.wikidata.org/entity/Q1055')]
[('location', 'http://www.wikidata.

30

### See how many "location" wdt:P276 and "place of pubblication" wdt:P291

In [6]:
queryString = """
SELECT (COUNT(?location) as ?numb1) (COUNT(?publ) AS ?numb2)
WHERE { 

    ?s wdt:P31/wdt:P279* wd:Q131569 .
    
    OPTIONAL{ ?s wdt:P276 ?location . }
    OPTIONAL{ ?s wdt:P291 ?publ . }
    
}
"""

print("Results")
run_query(queryString)

Results
[('numb1', '732'), ('numb2', '11')]


1

### Search for locations

In [7]:
queryString = """
SELECT DISTINCT ?location ?locName
WHERE { 

    ?s wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P276 ?location .
    ?location sc:name ?locName .
} LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('location', 'http://www.wikidata.org/entity/Q1187087'), ('locName', 'Florida Territory')]
[('location', 'http://www.wikidata.org/entity/Q464955'), ('locName', 'Windsor')]
[('location', 'http://www.wikidata.org/entity/Q246884'), ('locName', 'Lewes')]
[('location', 'http://www.wikidata.org/entity/Q583982'), ('locName', 'Anjangaon')]
[('location', 'http://www.wikidata.org/entity/Q672990'), ('locName', 'Bilhorod-Dnistrovskyi')]
[('location', 'http://www.wikidata.org/entity/Q71279'), ('locName', 'Malbork Castle')]
[('location', 'http://www.wikidata.org/entity/Q605984'), ('locName', 'Amapala')]
[('location', 'http://www.wikidata.org/entity/Q1663'), ('locName', 'Split')]
[('location', 'http://www.wikidata.org/entity/Q393'), ('locName', 'Szczecin')]
[('location', 'http://www.wikidata.org/entity/Q4120832'), ('locName', 'Königsberg')]
[('location', 'http://www.wikidata.org/entity/Q41795540'), ('locName', 'The Hague')]
[('location', 'http://www.wikidata.org/entity/Q56037'), ('locName', 

20

### Search information about the entity Zürich

In [8]:
queryString = """
SELECT DISTINCT ?p ?pName ?oName
WHERE { 

    wd:Q72 ?p ?o .
    OPTIONAL{?p sc:name ?pName}
    OPTIONAL{?o sc:name ?oName}
    
} ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1566')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1617')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1667')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2347')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2581')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P268')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P269')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P402')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P5063')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P6821')]
[('p', 'http://www.wikidata.org/prop/direct-

171

### Search information about Italy

In [9]:
queryString = """
SELECT DISTINCT ?p ?pName ?oName
WHERE { 

    wd:Q38 ?p ?o .
    OPTIONAL{?p sc:name ?pName}
    OPTIONAL{?o sc:name ?oName}
    
} ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1566')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1617')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1667')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2163')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P227')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2347')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2581')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P2950')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P349')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3911')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3916')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P402')]
[('p', 'http://www.wikidata.org/prop/direc

497

### Retrieve also  European Union URI

In [10]:
queryString = """
SELECT DISTINCT ?oName ?o
WHERE { 

    wd:Q38 wdt:P463 ?o .
    OPTIONAL{?o sc:name ?oName .}
    FILTER regex(?oName,"^European Union")
    
} ORDER BY ?pName
"""

print("Results")
run_query(queryString)

Results
[('oName', 'European Union'), ('o', 'http://www.wikidata.org/entity/Q458')]


1

### Retrieve treaties that took place in E.U. countries

In [11]:
queryString = """
SELECT ?countryName (GROUP_CONCAT(DISTINCT ?treatyName; SEPARATOR="; ") AS ?treaties)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P276 ?location ;
       sc:name ?treatyName .
    ?location wdt:P17 ?country .
    ?country wdt:P463 wd:Q458 ;
        sc:name ?countryName .
} GROUP BY ?countryName
"""

print("Results")
run_query(queryString)

Results
[('countryName', 'Sweden'), ('treaties', 'Convention Establishing the World Intellectual Property Organization; Peace of Lund; Second Treaty of Brömsebro; Stockholm Convention on Persistent Organic Pollutants; Treaty of Orebro')]
[('countryName', 'Czech Republic'), ('treaties', 'New START; Peace of Olomouc; Peace of Prague; Reichstadt Agreement; peace of Prague')]
[('countryName', 'Portugal'), ('treaties', 'Convention of Sintra; Faro Convention; Iberian Pact; Lisbon Protocol; Methuen Treaty; Treaty of Alcáçovas; Treaty of Lisbon')]
[('countryName', 'Lithuania'), ('treaties', 'Treaty of Kėdainiai; Truce of Vilna')]
[('countryName', 'Greece'), ('treaties', 'Prespa agreement; Protocol of Corfu; Salonika Agreement; Treaty of Accession 1979; Treaty of Accession 2003')]
[('countryName', 'Bulgaria'), ('treaties', 'Treaty of Küçük Kaynarca')]
[('countryName', 'United Kingdom'), ('treaties', 'Agreement Concerning the Shipwrecked Vessel RMS Titanic; Agreement on German External Debts; An

24

### Results part 1
All the treaties done in E.U. contry have been retrieved and grouped by the country where they are located

## Part 2
### Identify the BGP for obtaining which countries or people signed the treaties

### Try to identify predicate who signed the treaties

In [12]:
queryString = """
SELECT DISTINCT ?pName ?p
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 .
    ?s ?p ?treaty .  
    OPTIONAL{ ?p sc:name ?pName .}  
} 
"""

print("Results")
run_query(queryString)

Results
[('pName', 'organization directed by the office or position'), ('p', 'http://www.wikidata.org/prop/direct/P2389')]
[('pName', 'repealed by'), ('p', 'http://www.wikidata.org/prop/direct/P2568')]
[('pName', 'objective of project or action'), ('p', 'http://www.wikidata.org/prop/direct/P3712')]
[('pName', 'amended by'), ('p', 'http://www.wikidata.org/prop/direct/P2567')]
[('pName', 'cites work'), ('p', 'http://www.wikidata.org/prop/direct/P2860')]
[('pName', 'advertises'), ('p', 'http://www.wikidata.org/prop/direct/P6291')]
[('pName', 'applies to jurisdiction'), ('p', 'http://www.wikidata.org/prop/direct/P1001')]
[('pName', 'field of work'), ('p', 'http://www.wikidata.org/prop/direct/P101')]
[('pName', 'product or material produced'), ('p', 'http://www.wikidata.org/prop/direct/P1056')]
[('pName', 'employer'), ('p', 'http://www.wikidata.org/prop/direct/P108')]
[('pName', 'founded by'), ('p', 'http://www.wikidata.org/prop/direct/P112')]
[('pName', 'political ideology'), ('p', 'http:/

89

### Try another way to reach the previous goal

In [13]:
queryString = """
SELECT DISTINCT ?pName ?p
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
        ?p ?o .
    OPTIONAL{ ?p sc:name ?pName .}  
} 
"""

print("Results")
run_query(queryString)

Results
[('pName', 'depositor'), ('p', 'http://www.wikidata.org/prop/direct/P2058')]
[('pName', 'repealed by'), ('p', 'http://www.wikidata.org/prop/direct/P2568')]
[('pName', 'gesetze-im-internet.de ID'), ('p', 'http://www.wikidata.org/prop/direct/P7677')]
[('pName', 'AustLII ID'), ('p', 'http://www.wikidata.org/prop/direct/P5799')]
[('pName', 'China Treaty Database ID'), ('p', 'http://www.wikidata.org/prop/direct/P8609')]
[('pName', 'amended by'), ('p', 'http://www.wikidata.org/prop/direct/P2567')]
[('pName', 'prohibits'), ('p', 'http://www.wikidata.org/prop/direct/P8739')]
[('pName', 'Politiklexikon ID'), ('p', 'http://www.wikidata.org/prop/direct/P9437')]
[('pName', 'United Nations Treaty Series Registration Number'), ('p', 'http://www.wikidata.org/prop/direct/P4231')]
[('pName', 'CELEX number'), ('p', 'http://www.wikidata.org/prop/direct/P476')]
[('pName', 'applies to jurisdiction'), ('p', 'http://www.wikidata.org/prop/direct/P1001')]
[('pName', 'number of participants'), ('p', 'ht

393

### Find treaties signatory wdt:P1891

In [14]:
queryString = """
SELECT ?treatyName (GROUP_CONCAT(DISTINCT ?signatoryName; SEPARATOR="; ") AS ?signatories)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P1891 ?signatory ;
       sc:name ?treatyName .
    ?signatory sc:name ?signatoryName
    
} GROUP BY ?treatyName
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('treatyName', 'Chanyuan Treaty'), ('signatories', 'Liao dynasty; Northern Song Dynasty')]
[('treatyName', 'Treaty of Athis-sur-Orge'), ('signatories', 'County of Flanders')]
[('treatyName', 'Treaty of Melno'), ('signatories', 'Grand Duchy of Lithuania; Kingdom of Poland; Teutonic Order')]
[('treatyName', 'British-Brazilian Treaty of 1826'), ('signatories', 'Empire of Brazil; United Kingdom of Great Britain and Ireland')]
[('treatyName', 'Nice Agreement Concerning the International Classification of Goods and Services for the Purposes of the Registration of Marks'), ('signatories', 'Austria; Belgium; Denmark; France; Germany; Hungary; Italy; Lebanon; Liechtenstein; Luxembourg; Monaco; Morocco; Netherlands; Norway; Portugal; Romania; Spain; Sweden; Switzerland; Tunisia; Turkey; United Kingdom')]
[('treatyName', 'General Security Agreement between the United Kingdom of Great Britain and Northern Ireland and the Kingdom of Spain concerning the protection of classified information

25

### Results part 2
Found all people/country signed the treaties and grouped by treaty. Limit the returned treaties to 25.

## Part 3
### How many treaties have been published per year?

### Try to retrieve date of treaties
[('pName', 'publication date'), ('p', 'http://www.wikidata.org/prop/direct/P577')]

In [15]:
queryString = """
SELECT ?treatyName ?date
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName .

    
} GROUP BY ?treatyName
LIMIT 25
"""

print("Results")
run_query(queryString)


Results
[('treatyName', 'Treaty on the Adoption of the Republic of Crimea to Russia'), ('date', '2014-03-18T00:00:00Z')]
[('treatyName', 'Convention on the Political Rights of Women'), ('date', '1952-12-20T00:00:00Z')]
[('treatyName', '1991 Sino-Soviet Border Agreement'), ('date', '1991-05-16T00:00:00Z')]
[('treatyName', 'Sino-Soviet Treaty of Friendship, Alliance and Mutual Assistance'), ('date', '1950-02-14T00:00:00Z')]
[('treatyName', 'Paris Peace Accords'), ('date', '1973-01-27T00:00:00Z')]
[('treatyName', 'Lahore Declaration'), ('date', '1999-02-21T00:00:00Z')]
[('treatyName', 'Secret Additional Protocol to the Molotov–Ribbentrop Pact'), ('date', '1939-08-23T00:00:00Z')]
[('treatyName', 'Convention on the Elimination of All Forms of Discrimination against Women'), ('date', '1979-12-18T00:00:00Z')]
[('treatyName', 'Agreement Concerning the Shipwrecked Vessel RMS Titanic'), ('date', '2003-01-01T00:00:00Z')]
[('treatyName', 'International convention regading cholera'), ('date', '1894

25

### Check if date is stored as literal

In [16]:
queryString = """
ASK WHERE { 

    FILTER isLiteral(?date)
    
    {
        SELECT ?date WHERE{
        ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
           wdt:P577 ?date ;
           sc:name ?treatyName .
        }
    }
    
} 
"""

print("Results")
run_ask_query(queryString)


Results


{'head': {'link': []}, 'boolean': True}

### Group treaties by year

In [17]:
queryString = """

SELECT ?year (COUNT(DISTINCT ?treaty) AS ?treatyNumb)
WHERE { 
    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date .
    BIND (year(xsd:dateTime(?date)) as ?year)
    
} GROUP BY ?year
ORDER BY DESC(?treatyNumb) DESC(?year)
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('year', '1929'), ('treatyNumb', '20')]
[('year', '1926'), ('treatyNumb', '18')]
[('year', '1924'), ('treatyNumb', '18')]
[('year', '1931'), ('treatyNumb', '17')]
[('year', '2000'), ('treatyNumb', '16')]
[('year', '1932'), ('treatyNumb', '16')]
[('year', '1936'), ('treatyNumb', '13')]
[('year', '1927'), ('treatyNumb', '11')]
[('year', '1928'), ('treatyNumb', '9')]
[('year', '1922'), ('treatyNumb', '9')]
[('year', '1938'), ('treatyNumb', '8')]
[('year', '1933'), ('treatyNumb', '8')]
[('year', '1999'), ('treatyNumb', '7')]
[('year', '1998'), ('treatyNumb', '7')]
[('year', '1935'), ('treatyNumb', '7')]


15

### Results part 3
Treaties grouped by year and counted.
We can see that (in our database) the largest number of treaties were made during the 20th century.

## Part 4
For how many treaties we know causes or effects?

### Search effects or causes predicates

In [18]:
queryString = """

SELECT DISTINCT ?pName ?p
WHERE { 
    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       ?p ?o .
    ?p sc:name ?pName .
    filter (regex(?pName, ".*[Cc]ause.*")||regex(?pName, ".*[Ee]ffect.*"))
}
"""

print("Results")
run_query(queryString)

Results
[('pName', 'has effect'), ('p', 'http://www.wikidata.org/prop/direct/P1542')]
[('pName', 'has cause'), ('p', 'http://www.wikidata.org/prop/direct/P828')]
[('pName', 'effective date'), ('p', 'http://www.wikidata.org/prop/direct/P7588')]
[('pName', 'has immediate cause'), ('p', 'http://www.wikidata.org/prop/direct/P1478')]


4

### Count how many treaties we know the couses or effects

In [19]:
queryString = """

SELECT COUNT(DISTINCT ?treaty)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
            (wdt:P1542|wdt:P828) ?event .
}
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '53')]


1

### Not required: see couses or effects (collected together)

In [20]:
queryString = """

SELECT ?treatyName (GROUP_CONCAT(DISTINCT ?eventName; SEPARATOR="; ") AS ?eventsName)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
            (wdt:P1542|wdt:P828) ?event ;
            sc:name ?treatyName.
    ?event sc:name ?eventName .
} GROUP BY ?treatyName
"""

print("Results")
run_query(queryString)

Results
[('treatyName', 'Congress of Vienna'), ('eventsName', 'Border Kingdom of Saxony-Kingdom of Prussia 1815; borderstones Belgium-Prussia')]
[('treatyName', 'Chapter VII of the United Nations Charter'), ('eventsName', 'peace enforcement')]
[('treatyName', 'Treaty of Trianon'), ('eventsName', 'Paris Peace Conference')]
[('treatyName', 'Munich agreement'), ('eventsName', 'Realpolitik; Sudeten Crisis')]
[('treatyName', 'Treaty of Saint-Germain-en-Laye'), ('eventsName', 'Paris Peace Conference')]
[('treatyName', 'Treaty of Sèvres'), ('eventsName', 'Paris Peace Conference')]
[('treatyName', 'Molotov–Ribbentrop Pact'), ('eventsName', 'Soviet invasion of Poland; Winter War')]
[('treatyName', 'Brioni Agreement'), ('eventsName', 'Ten-Day War')]
[('treatyName', 'Boundary Treaty of 1866 between Chile and Bolivia'), ('eventsName', 'Lindsay-Corral agreement')]
[('treatyName', 'Treaty of Hubertusburg'), ('eventsName', 'Seven Years War')]
[('treatyName', 'Anglo-Russian Entente'), ('eventsName', '

43

### Results part 4
In our database there are only 53 treaties with known causes or effects. When I decided to retreive also the eventsName we noticed that the event is known but not its name. In particular I retrieved only 43 treaties probably because in the database some treaties names are missing.

## Part 5

Analyze the number of countries and how many treaties they have signed and their causes

### Part 5.1
Are there two countries that signed many common treaties?

In [21]:
queryString = """

SELECT DISTINCT ?sign1Name ?sign2Name (COUNT(DISTINCT ?treaty) AS ?numbCommonTreaties)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory1 .
    ?signatory1 sc:name ?sign1Name .
    
    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory2 .
     ?signatory2 sc:name ?sign2Name .
    
    FILTER (?signatory1 != ?signatory2)
    
} GROUP BY ?sign1Name ?sign2Name
ORDER BY DESC(?numbCommonTreaties)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('sign1Name', 'Canada'), ('sign2Name', 'Australia'), ('numbCommonTreaties', '3')]
[('sign1Name', 'United Kingdom'), ('sign2Name', 'United States of America'), ('numbCommonTreaties', '3')]
[('sign1Name', 'Australia'), ('sign2Name', 'Canada'), ('numbCommonTreaties', '3')]
[('sign1Name', 'United States of America'), ('sign2Name', 'United Kingdom'), ('numbCommonTreaties', '3')]
[('sign1Name', 'Australia'), ('sign2Name', 'Russia'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Panama'), ('sign2Name', 'Nicaragua'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Ireland'), ('sign2Name', 'Philippines'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Fiji'), ('sign2Name', 'Brazil'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Trinidad and Tobago'), ('sign2Name', 'Nigeria'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Fiji'), ('sign2Name', 'Kazakhstan'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Liechtenstein'), ('sign2Name', 'Zambia'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Franc

25

### Try to remove duplicate pair

In [22]:
queryString = """

SELECT DISTINCT ?sign1Name ?sign2Name (COUNT(DISTINCT ?treaty) AS ?numbCommonTreaties)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory1 .
    ?signatory1 sc:name ?sign1Name .
    
    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory2 .
     ?signatory2 sc:name ?sign2Name .
    
    FILTER (str(?signatory1) < str(?signatory2))
    
} GROUP BY ?sign1Name ?sign2Name
ORDER BY DESC(?numbCommonTreaties)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('sign1Name', 'United Kingdom'), ('sign2Name', 'United States of America'), ('numbCommonTreaties', '3')]
[('sign1Name', 'Canada'), ('sign2Name', 'Australia'), ('numbCommonTreaties', '3')]
[('sign1Name', 'Kazakhstan'), ('sign2Name', 'Ireland'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Austria'), ('sign2Name', 'Ecuador'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Holy See'), ('sign2Name', 'El Salvador'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Venezuela'), ('sign2Name', 'Bangladesh'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Venezuela'), ('sign2Name', 'Philippines'), ('numbCommonTreaties', '2')]
[('sign1Name', 'France'), ('sign2Name', 'Bolivia'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Bolivia'), ('sign2Name', 'Saint Vincent and the Grenadines'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Malta'), ('sign2Name', 'Panama'), ('numbCommonTreaties', '2')]
[('sign1Name', 'France'), ('sign2Name', 'Ireland'), ('numbCommonTreaties', '2')]
[('sign1Name', 'Liechtenst

25

### Result part 5.1
Finally I have found (unique) pair of country taht signed common treaties. 

### Part 5.2

#### Which are the treaties with most signatories?

In [23]:
queryString = """

SELECT ?treatyName (COUNT(DISTINCT ?signatory) AS ?signatoryNumb)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory .
    
} GROUP BY ?treatyName
ORDER BY DESC(?signatoryNumb) 
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('treatyName', 'Universal Copyright Convention'), ('signatoryNumb', '97')]
[('treatyName', 'Treaty on the Prohibition of Nuclear Weapons'), ('signatoryNumb', '86')]
[('treatyName', 'North Atlantic Treaty'), ('signatoryNumb', '23')]
[('treatyName', 'Paris Agreement'), ('signatoryNumb', '14')]
[('treatyName', 'International convention regading cholera'), ('signatoryNumb', '11')]
[('treatyName', 'Grenelle agreements'), ('signatoryNumb', '8')]
[('treatyName', 'Anzus'), ('signatoryNumb', '6')]
[('treatyName', 'international yard and pound agreement'), ('signatoryNumb', '6')]
[('treatyName', 'Treaty of Brest-Litovsk'), ('signatoryNumb', '5')]
[('treatyName', 'Munich agreement'), ('signatoryNumb', '4')]
[('treatyName', 'North Pacific Fur Seal Convention of 1911'), ('signatoryNumb', '4')]
[('treatyName', 'EU–UK Trade and Cooperation Agreement'), ('signatoryNumb', '3')]
[('treatyName', 'Treaty of Vienna'), ('signatoryNumb', '3')]
[('treatyName', 'Korean Armistice Agreement'), ('signato

15

#### Which country signed the most treaties?

In [24]:
queryString = """

SELECT ?signatoryName (COUNT(DISTINCT ?treaty) AS ?treatyNumb)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory .
    
    ?signatory sc:name ?signatoryName ;
               wdt:P31 wdt:P17 .
    
} GROUP BY ?signatoryName
ORDER BY DESC(?treatyNumb) 
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
Empty


0

### Adjust the previous query (signatory triples)

In [25]:
queryString = """

SELECT ?signatoryName (COUNT(DISTINCT ?treaty) AS ?treatyNumb)
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory .
    
    ?signatory sc:name ?signatoryName ;
        wdt:P17 ?signatory .
    
} GROUP BY ?signatoryName 
ORDER BY DESC(?treatyNumb) 
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('signatoryName', 'United Kingdom'), ('treatyNumb', '5')]
[('signatoryName', 'United States of America'), ('treatyNumb', '5')]
[('signatoryName', 'France'), ('treatyNumb', '4')]
[('signatoryName', 'India'), ('treatyNumb', '3')]
[('signatoryName', 'Canada'), ('treatyNumb', '3')]
[('signatoryName', 'Australia'), ('treatyNumb', '3')]
[('signatoryName', 'Russia'), ('treatyNumb', '3')]
[('signatoryName', 'Soviet Union'), ('treatyNumb', '3')]
[('signatoryName', 'New Zealand'), ('treatyNumb', '3')]
[('signatoryName', 'South Africa'), ('treatyNumb', '2')]
[('signatoryName', 'Czech Republic'), ('treatyNumb', '2')]
[('signatoryName', 'Panama'), ('treatyNumb', '2')]
[('signatoryName', 'Philippines'), ('treatyNumb', '2')]
[('signatoryName', 'Brazil'), ('treatyNumb', '2')]
[('signatoryName', 'South Korea'), ('treatyNumb', '2')]


15

### Result part 5.2
We can see from the above results that the UK and US are the country with the most number of signed treaties

### Part 5.3
#### Are there countries that did not sign a treaty with the U.S. but signed a treaty with Germany?

### Search country URI

In [26]:
queryString = """

SELECT DISTINCT ?country ?countryName
WHERE { 

    ?s wdt:P17 ?country .
    ?country sc:name ?countryName .
    
    FILTER (regex(?countryName,"^[Gg]ermany")||regex(?countryName,".*[Ss]tates.*[Aa]merica.*"))
       
} LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America')]
[('country', 'http://www.wikidata.org/entity/Q81931'), ('countryName', 'Confederate States of America')]
[('country', 'http://www.wikidata.org/entity/Q19971019'), ('countryName', 'United States of America')]
[('country', 'http://www.wikidata.org/entity/Q99281400'), ('countryName', 'United States of America')]


5

In [27]:
queryString = """

SELECT DISTINCT ?pName ?oName ?p ?o
WHERE { 

    wd:Q30 sc:name ?countryName ;
        ?p ?o .
    OPTIONAL{?p sc:name ?pName .}
    OPTIONAL{?o sc:name ?oName .}
       
} LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('pName', "topic's main Wikimedia portal"), ('oName', 'Portal:United States'), ('p', 'http://www.wikidata.org/prop/direct/P1151'), ('o', 'http://www.wikidata.org/entity/Q5365167')]
[('pName', 'basic form of government'), ('oName', 'presidential régime'), ('p', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'http://www.wikidata.org/entity/Q49892')]
[('pName', 'basic form of government'), ('oName', 'federal republic'), ('p', 'http://www.wikidata.org/prop/direct/P122'), ('o', 'http://www.wikidata.org/entity/Q512187')]
[('pName', 'central bank'), ('oName', 'Federal Reserve System'), ('p', 'http://www.wikidata.org/prop/direct/P1304'), ('o', 'http://www.wikidata.org/entity/Q53536')]
[('pName', 'office held by head of government'), ('oName', 'President of the United States'), ('p', 'http://www.wikidata.org/prop/direct/P1313'), ('o', 'http://www.wikidata.org/entity/Q11696')]
[('pName', 'described by source'), ('oName', '1922 Encyclopædia Britannica'), ('p', 'http://www.wikidata.or

10

### Use "istance of" predicate

In [28]:
queryString = """

SELECT DISTINCT ?country ?countryName ?oName ?o
WHERE { 

    ?s wdt:P17 ?country .
    ?country sc:name ?countryName ;
            wdt:P31 ?o .
    OPTIONAL{?o sc:name ?oName .}
    
    FILTER (regex(?countryName,".*[Ss]tates.*[Aa]merica.*"))
       
}
"""

print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('oName', 'superpower'), ('o', 'http://www.wikidata.org/entity/Q1489259')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('oName', 'historical unrecognized state'), ('o', 'http://www.wikidata.org/entity/Q99541706')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('oName', 'constitutional republic'), ('o', 'http://www.wikidata.org/entity/Q1520223')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('oName', 'sovereign state'), ('o', 'http://www.wikidata.org/entity/Q3624078')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of America'), ('oName', 'federal republic'), ('o', 'http://www.wikidata.org/entity/Q512187')]
[('country', 'http://www.wikidata.org/entity/Q30'), ('countryName', 'United States of Ame

11

### Treaties not signed by U.S. but signed by Germany

In [29]:
queryString = """

SELECT DISTINCT ?treatyName
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory .
    
    FILTER EXISTS {?treaty wdt:P1891 wd:Q183 .}
    FILTER NOT EXISTS {?treaty wdt:P1891 wd:Q30 .}
    
} GROUP BY ?signatoryName 
ORDER BY DESC(?treatyNumb) 
LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('treatyName', 'Paris Agreement')]
[('treatyName', 'Kyoto Protocol')]


2

### Countries that signed the above treaties

In [30]:
queryString = """

SELECT DISTINCT ?signatoryName
WHERE { 

    ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
       wdt:P577 ?date ;
       sc:name ?treatyName ;
       wdt:P1891 ?signatory .
       
    ?signatory sc:name ?signatoryName
    
    FILTER EXISTS {?treaty wdt:P1891 wd:Q183 .}
    FILTER NOT EXISTS {?treaty wdt:P1891 wd:Q30 .}
    
}
"""

print("Results")
run_query(queryString)

Results
[('signatoryName', 'Guinea')]
[('signatoryName', 'Niger')]
[('signatoryName', 'Seychelles')]
[('signatoryName', 'France')]
[('signatoryName', "People's Republic of China")]
[('signatoryName', 'Brazil')]
[('signatoryName', 'Russia')]
[('signatoryName', 'Canada')]
[('signatoryName', 'Japan')]
[('signatoryName', 'Germany')]
[('signatoryName', 'Estonia')]
[('signatoryName', 'Norway')]
[('signatoryName', 'Latvia')]
[('signatoryName', 'Ukraine')]
[('signatoryName', 'Czech Republic')]
[('signatoryName', 'Slovakia')]
[('signatoryName', 'Romania')]
[('signatoryName', 'Bulgaria')]
[('signatoryName', 'Croatia')]
[('signatoryName', 'Andorra')]
[('signatoryName', 'Kazakhstan')]
[('signatoryName', 'Malta')]
[('signatoryName', 'Monaco')]
[('signatoryName', 'Indonesia')]
[('signatoryName', 'Uzbekistan')]
[('signatoryName', 'Ireland')]
[('signatoryName', 'Spain')]
[('signatoryName', 'Chile')]
[('signatoryName', 'Kingdom of the Netherlands')]
[('signatoryName', 'Belgium')]
[('signatoryName', 'Lu

47

### Countries that did not sign a treaty with the U.S. but signed a treaty with Germany

In [31]:
queryString = """

SELECT DISTINCT ?signatoryName
WHERE { 

    {
        SELECT ?signatoryName
        WHERE{
            ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
               wdt:P577 ?date ;
               sc:name ?treatyName ;
               wdt:P1891 ?signatory .
            ?signatory sc:name ?signatoryName
            FILTER EXISTS {?treaty wdt:P1891 wd:Q183 .}
        }
    }
    MINUS
    {
        SELECT ?signatoryName
        WHERE{
            ?treaty wdt:P31/wdt:P279* wd:Q131569 ;
               wdt:P577 ?date ;
               sc:name ?treatyName ;
               wdt:P1891 ?signatory .
            ?signatory sc:name ?signatoryName
            FILTER EXISTS {?treaty wdt:P1891 wd:Q30 .}
        }
    }
}
"""

print("Results")
run_query(queryString)

Results
[('signatoryName', 'Seychelles')]
[('signatoryName', 'Japan')]
[('signatoryName', 'Romania')]
[('signatoryName', 'Indonesia')]
[('signatoryName', 'Kingdom of the Netherlands')]
[('signatoryName', 'Niue')]
[('signatoryName', 'Greece')]
[('signatoryName', 'Turkey')]
[('signatoryName', 'Tuvalu')]
[('signatoryName', 'Federated States of Micronesia')]
[('signatoryName', 'Marshall Islands')]
[('signatoryName', 'Saint Lucia')]
[('signatoryName', 'Antigua and Barbuda')]
[('signatoryName', 'Honduras')]
[('signatoryName', 'Nicaragua')]
[('signatoryName', 'Maldives')]
[('signatoryName', 'Thailand')]
[('signatoryName', 'Vietnam')]
[('signatoryName', 'Bangladesh')]
[('signatoryName', 'Mali')]


20

### Result part 5.3
We can conclude that 20 country never signed a treaty with U.S. (at least in our database) but the signed at least a treaty with Germany